In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report,mean_squared_error, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedBaggingClassifier
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [2]:
# put the dataset which comes from short_dataset_feature_engineering.ipynb 
file_path = '/data/caysar9/results/final_short.csv'
df = pd.read_csv(file_path)

In [3]:
# predicting the next migraine in the next day

features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']

target = 'next_migraine_next_day'  

X = df[features]

y = df[target]
  

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Frequency counts before applying SMOTE
print("Frequency counts before SMOTE:")
print("Frequency counts train set")
print(y_train.value_counts())
print("Frequency counts test set")
print(y_test.value_counts())

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
# Frequency counts before applying SMOTE
print("Frequency counts after SMOTE:")
print("Frequency counts train set")
print(y_train_balanced.value_counts())

Frequency counts before SMOTE:
Frequency counts train set
next_migraine_next_day
0    10354
1     1167
Name: count, dtype: int64
Frequency counts test set
next_migraine_next_day
0    2589
1     292
Name: count, dtype: int64
Frequency counts after SMOTE:
Frequency counts train set
next_migraine_next_day
0    10354
1    10354
Name: count, dtype: int64


In [7]:
# Define the hyperparameter distributions
param_distributions = {
    'penalty': ['l1', 'l2', 'elasticnet'],  # Regularization methods
    'C': uniform(0.01, 10),  # Continuous uniform distribution for regularization strength
    'solver': ['liblinear', 'saga'],  # solvers
    'max_iter': randint(100, 1000)  # Randomly sample number of iterations
}

# Initialize the Logistic Regression model
lr_model = LogisticRegression(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=param_distributions,
    n_iter=30,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluate based on AUC
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits


/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/s

Best Hyperparameters: {'C': 6.126531604882809, 'max_iter': 352, 'penalty': 'l1', 'solver': 'liblinear'}

AUC Score: 0.5606

Confusion Matrix:
[[1334 1255]
 [ 127  165]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.52      0.66      2589
           1       0.12      0.57      0.19       292

    accuracy                           0.52      2881
   macro avg       0.51      0.54      0.43      2881
weighted avg       0.83      0.52      0.61      2881



In [8]:
X_train_balanced_const = sm.add_constant(X_train_balanced)

# Fit the logistic regression model on the balanced data
logit_model = sm.Logit(y_train_balanced, X_train_balanced_const).fit()

# Display the summary of the model
print(logit_model.summary())


Optimization terminated successfully.
         Current function value: 0.613602
         Iterations 7
                             Logit Regression Results                             
Dep. Variable:     next_migraine_next_day   No. Observations:                20708
Model:                              Logit   Df Residuals:                    20695
Method:                               MLE   Df Model:                           12
Date:                    Sat, 07 Dec 2024   Pseudo R-squ.:                  0.1148
Time:                            20:01:28   Log-Likelihood:                -12706.
converged:                           True   LL-Null:                       -14354.
Covariance Type:                nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                           

In [9]:
# Define the hyperparameter distributions
param_distributions = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # Splitting criteria
    'max_depth': randint(5, 50),  # Randomly sample tree depth between 5 and 50
    'min_samples_split': randint(2, 20),  # Minimum samples required to split a node
    'min_samples_leaf': randint(1, 10),  # Minimum samples required in a leaf node
    'max_features': [None, 'sqrt', 'log2']  # Number of features to consider for best split
}

# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt_model,
    param_distributions=param_distributions,
    n_iter=30,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluate based on AUC
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 26, 'max_features': None, 'min_samples_leaf': 9, 'min_samples_split': 18}

AUC Score: 0.5065

Confusion Matrix:
[[2207  382]
 [ 250   42]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.85      0.87      2589
           1       0.10      0.14      0.12       292

    accuracy                           0.78      2881
   macro avg       0.50      0.50      0.50      2881
weighted avg       0.82      0.78      0.80      2881



In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from scipy.stats import randint

# Define the hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 300),  # Randomly sample between 50 and 300 trees
    'max_depth': [None, 10, 20, 30],  # Fixed choices for tree depth
    'min_samples_split': randint(2, 11),  # Randomly sample between 2 and 10 for splitting
    'min_samples_leaf': randint(1, 5),  # Randomly sample between 1 and 4 for leaf nodes
    'max_features': ['sqrt', 'log2', None],  # Fixed options for max features
    'bootstrap': [True, False]  # Fixed options for bootstrapping
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluate based on AUC
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


Best Hyperparameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 267}

AUC Score: 0.5392

Confusion Matrix:
[[2377  212]
 [ 267   25]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2589
           1       0.11      0.09      0.09       292

    accuracy                           0.83      2881
   macro avg       0.50      0.50      0.50      2881
weighted avg       0.82      0.83      0.83      2881



In [11]:
# Get feature importances from the trained Random Forest model
feature_importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X_train_balanced.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:\n")
print(feature_importance_df)


Feature Importances:

                             Feature  Importance
7         migraine_attacks_past7days    0.263546
8   mean_migraine_duration_past7days    0.149300
6                                age    0.127175
5         sleep_duration_past_7_days    0.119795
4               sleep_duration_hours    0.116856
9            total_physical_activity    0.096041
2                 trigger_poor_sleep    0.032532
3                     trigger_stress    0.029963
11               reported_depression    0.025399
10                  reported_anxiety    0.018856
0     trigger_lack_physical_activity    0.015625
1          trigger_physical_activity    0.004912


In [12]:
# Define the hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 200),  # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),  # Learning rate for boosting
    'max_depth': randint(3, 10),  # Maximum depth of individual estimators
    'min_samples_split': randint(2, 10),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 5),  # Minimum number of samples required to be a leaf node
    'subsample': uniform(0.7, 0.3),  # Fraction of samples used for fitting each base learner
    'max_features': ['sqrt', 'log2', None]  # Number of features to consider at each split
}

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=30,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluate based on AUC
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best Hyperparameters: {'learning_rate': 0.268219174976903, 'max_depth': 9, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 186, 'subsample': 0.8454489914076949}

AUC Score: 0.5312

Confusion Matrix:
[[2380  209]
 [ 266   26]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2589
           1       0.11      0.09      0.10       292

    accuracy                           0.84      2881
   macro avg       0.51      0.50      0.50      2881
weighted avg       0.82      0.84      0.83      2881



In [13]:
# Get feature importances from the trained Gradient boosting model
feature_importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X_train_balanced.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:\n")
print(feature_importance_df)


Feature Importances:

                             Feature  Importance
7         migraine_attacks_past7days    0.351541
8   mean_migraine_duration_past7days    0.121052
4               sleep_duration_hours    0.100585
5         sleep_duration_past_7_days    0.097222
6                                age    0.094280
9            total_physical_activity    0.094027
11               reported_depression    0.036398
3                     trigger_stress    0.036379
2                 trigger_poor_sleep    0.032581
10                  reported_anxiety    0.017640
0     trigger_lack_physical_activity    0.013222
1          trigger_physical_activity    0.005073


In [14]:
# Define the hyperparameter distributions
param_distributions = {
    'n_neighbors': randint(1, 50),  # Number of neighbors to test
    'weights': ['uniform', 'distance'],  # Weighting strategies
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Distance metrics
    'p': randint(1, 3)  # distance parameter 
}

# Initialize the K-Nearest Neighbors model
knn_model = KNeighborsClassifier()

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn_model,
    param_distributions=param_distributions,
    n_iter=30,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluate based on AUC
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best Hyperparameters: {'metric': 'manhattan', 'n_neighbors': 6, 'p': 2, 'weights': 'distance'}

AUC Score: 0.5362
Confusion Matrix:
[[1677  912]
 [ 171  121]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.65      0.76      2589
           1       0.12      0.41      0.18       292

    accuracy                           0.62      2881
   macro avg       0.51      0.53      0.47      2881
weighted avg       0.83      0.62      0.70      2881



In [15]:
'''from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scikeras.wrappers import KerasClassifier
import numpy as np

# Define features and target
features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']
target = 'next_migraine_next_day'

X = df[features]
y = df[target]


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Define a function to create the model
def create_model(learning_rate=0.001, neurons=[64, 32, 16]):
    model = Sequential()
    model.add(Dense(neurons[0], activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(neurons[1], activation='relu'))
    model.add(Dense(neurons[2], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model for compatibility with RandomizedSearchCV
keras_clf = KerasClassifier(model=create_model, verbose=0)

# Define hyperparameter distributions
param_distributions = {
    "model__learning_rate": [0.001, 0.01, 0.1],
    "model__neurons": [[64, 32, 16], [128, 64, 32], [32, 16, 8]],
    "batch_size": [16, 32, 64],
    "epochs": [20, 50, 100]
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=keras_clf,
    param_distributions=param_distributions,
    n_iter=10,  # Number of random combinations to try
    scoring='roc_auc',  # Focus on auc for hyperparameter tuning
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Perform hyperparameter tuning
random_search.fit(X_train_scaled, y_train_smote)

# Get the best model and parameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate the model on the test set
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class
y_pred_classes = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary classes

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nAUC Score: {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))
'''

'\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense\nfrom tensorflow.keras.optimizers import Adam\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.metrics import roc_auc_score, confusion_matrix, classification_report\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import train_test_split\nfrom imblearn.over_sampling import SMOTE\nfrom scikeras.wrappers import KerasClassifier\nimport numpy as np\n\n# Define features and target\nfeatures = [\'trigger_lack_physical_activity\',\'trigger_physical_activity\',\'trigger_poor_sleep\',\'trigger_stress\',\'sleep_duration_hours\',\'sleep_duration_past_7_days\',\'age\',\'migraine_attacks_past7days\',\'mean_migraine_duration_past7days\',\'total_physical_activity\',\'reported_anxiety\', \'reported_depression\']\n\nX = df[features]\ny = df[\'affected_activity_QoL\']\n\n# Split data into training and test sets\nX_train, X_test, y_train, y_test = train_test_spl